In [1]:
import brightway2 as bw
import numpy as np
import pandas as pd
from copy import copy

# Setup the project

### TODO -> choose project

In [2]:
project = 'GSA for ecoinvent'

In [3]:
if project not in bw.projects:
    bw.projects.create_project(project)
bw.projects.set_current(project)

In [4]:
bw.projects.dir


'/Users/akim/Library/Application Support/Brightway3/GSA-for-ecoinvent.34cad902cfa956f8f1e5e24776ceebde'

# Import databases

### TODO -> give paths to databases files
for now let's use ecoinvent 3.3 cutoff and EXIOBASE 2.2

In [ ]:
ei_path = '/Users/akim/Documents/LCA files/ecoinvent 3.3 cutoff/datasets'  
ex_path = '/Users/akim/Documents/LCA files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
ag_path = '/Users/akim/Documents/LCA files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'

## 1. Biosphere3

In [ ]:
bw.bw2setup()

## 2. Ecoinvent 3.3

In [ ]:
ei_name = "ecoinvent 3.3 cutoff"  
if ei_name in bw.databases:
    print(ei_name + " database already present!!! No import is needed")
else:
    ei = bw.SingleOutputEcospold2Importer(ei_path, ei_name)
    ei.apply_strategies()
    ei.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei.statistics()

In [ ]:
if ei_name in bw.databases:
    print(ei_name + " database already present!!! No import is needed")
else:
    ei.write_database()

## 3. Exiobase 2.2

In [ ]:
# 2. Exiobase 
from bw2io.importers.exiobase import Exiobase22Importer

In [ ]:
ex_name = 'EXIOBASE 2.2'
if ex_name in bw.databases:
    print(ex_name + " database already present!!! No import is needed")
else:
    ex = Exiobase22Importer(ex_path, ex_name)
    ex.apply_strategies()

In [ ]:
if ex_name in bw.databases:
    print(ex_name + " database already present!!! No import is needed")
else:
    ex.write_database()

## 4. Agribalyse 1.2
better see: [how to import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%20with%20Ecoinvent%202.2.ipynb)

In [ ]:
ag_name = 'Agribalyse 1.2'
if ag_name in bw.databases:
    print(ag_name + " database already present!!! No import is needed")
else:
    ag = bw.SingleOutputEcospold1Importer(ag_path, ag_name)
    ag.apply_strategies()
    ag.statistics()

In [ ]:
if ag_name in bw.databases:
    print(ag_name + " database already present!!! No import is needed")
else:
    ag.write_excel(True)
    from bw2io.strategies.simapro import normalize_simapro_biosphere_categories, normalize_simapro_biosphere_names
    ag.apply_strategy(normalize_simapro_biosphere_categories)
    ag.apply_strategy(normalize_simapro_biosphere_names)

    from bw2io.strategies import link_iterable_by_fields
    import functools
    ag.apply_strategy(functools.partial(link_iterable_by_fields, other=bw.Database("biosphere3"), kind="biosphere"))
    ag.statistics()

    ag.write_excel(True)

    bw.Database("Agribalyse new biosphere").register()
    ag.add_unlinked_flows_to_biosphere_database("Agribalyse new biosphere")

    def get_unlinked(data):
        for ds in ag.data:
            for exc in ds['exchanges']:
                if exc['type'] == 'production' and exc['name'] == 'Alfalfa, conventional, for animal feeding, at farm gate':
                    return ds, exc

    ds, exc = get_unlinked(ag.data)

    from bw2io.strategies import link_technosphere_based_on_name_unit_location
    ag.apply_strategy(link_technosphere_based_on_name_unit_location)
    ag.statistics()

    def get_unlinked(data):
        for ds in ag.data:
            for exc in ds['exchanges']:
                if exc['type'] == 'production' and not exc.get('input'):
                    return ds, exc

    ds, exc = get_unlinked(ag.data)

    for ds in ag.data:
        for exc in ds['exchanges']:
            if exc['type'] == 'production' and not exc.get('input'):
                print("Fixing:", exc['name'])
                exc['type'] = 'technosphere'

    ag.apply_strategy(link_technosphere_based_on_name_unit_location)
    ag.apply_strategy(functools.partial(link_technosphere_based_on_name_unit_location, external_db_name="ecoinvent 2.2"))
    ag.statistics()

    ag.add_unlinked_activities()
    ag.statistics()

    for ds in ag.data:
        del ds['code']

    from bw2io.strategies import set_code_by_activity_hash
    ag.apply_strategy(set_code_by_activity_hash)

    ag.apply_strategy(functools.partial(
            link_iterable_by_fields,
            other=ag.data,
            fields=('name', 'location', 'unit'),
            relink=True
    ))
    ag.statistics()

In [ ]:
if ag_name in bw.databases:
    print(ag_name + " database already present!!! No import is needed")
else:
    ag.write_database()

## 5. HEIA

### TODO -> ask Andi for files

In [ ]:
bw.databases

# Create ecoinvent and exiobase LCA objects

In [ ]:
n_methods = 2
methods = {_: bw.methods.random() for _ in range(n_methods)}
methods

In [ ]:
# 1. Ecoinvent
ei_demand = {bw.Database(ei_name).random(): 1}
ei_lca = bw.LCA(ei_demand, methods[0])
ei_lca.lci()
ei_lca.lcia()
ei_lca.score

In [ ]:
# 2. Exiobase
ex_demand = {bw.Database(ex_name).random(): 1}
ex_lca = bw.LCA(ex_demand, methods[0])
ex_lca.lci()
ex_lca.lcia()
ex_lca.score

In [ ]:
methods

# Consumption model

In [ ]:
methods

In [ ]:
from consumption_copy import ConsumptionLCA

In [ ]:
# consumption_model_path = 'Draft_Paper_8_v11_SupportingInformation.xlsx'
# sheet_lca_modeling = 'Overview & LCA-Modeling'

consumption_model_path = 'Consumption_Cockpit_adj.xlsx'
sheet_lca_modeling = 'LCA-Modelling'

In [ ]:
excel = {
    'path': consumption_model_path,
    'sheet': sheet_lca_modeling
}

In [ ]:
a = ConsumptionLCA(ei_lca, ex_lca, methods, excel, archetypes=True)

# Extract consumption vector

In [ ]:
path = 'Draft_Paper_8_v11_SupportingInformation.xlsx'
df = pd.read_excel(path, sheet_name = 'Overview & LCA-Modeling', header=2)

In [ ]:
def complete_columns(df):
    
    column_names = list(df.columns)
    indices = [i for i,el in enumerate(column_names)  if 'Activity' in el]
    column_names_complete = copy(column_names)

    n_el_added = 0
    for ind in indices:
        if 'On' not in column_names[ind-1]:
            act_name = column_names[ind]
            act_number = act_name[act_name.find(' ')+1:]
            column_names_complete.insert(ind+n_el_added, 'On ' + act_number)
            n_el_added += 1
        
    df.columns = column_names_complete[:len(column_names)]
    
    return df

In [ ]:
path_complete = path
df.to_excel()

In [ ]:
bw.Database('ecoinvent 3.6 cutoff').search('rice production, non-basmati')

In [ ]:
column_names.insert?

## 1. Add `On's` into columns that Andi forgot to include

In [ ]:
ls

In [ ]:
from consumption_copy import ConsumptionLCA

In [ ]:
ei_lca

In [21]:
act = bw.Database('biosphere3').random()

In [23]:
act.as_dict()

{'categories': ('air', 'low population density, long-term'),
 'code': 'd482b568-0604-4433-87b2-1340f7a17512',
 'name': 'Methanol',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission'}